# Setup Kubernetes environment for DYNAMOS in FABRIC 

This Jupyter notebook will create the Kubernetes environment in FABRIC after the slice and corresponding nodes have been created.

FABRIC API docs: https://fabric-fablib.readthedocs.io/en/latest/index.html


## Step 1: Configure the Environment (has to be done once in the Jupyter Hub environment) & Create Slice

Before running this notebook, you will need to configure your environment using the [Configure Environment](../../configure_and_validate.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook. Note: this has to be done only once in the Jupyter Hub environment (unless configuration is removed/deleted of course).

If you are using the FABRIC JupyterHub many of the environment variables will be automatically configured for you.  You will still need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  

After following all steps of the Configuring Environment notebook, you should be able to run this notebook without additional steps.

Next, you will need to have setup the slice in FABRIC using the [Create Slice](../../create_slice.ipynb) notebook.

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).
 

## Step 2: Setup the Environment for this Notebook

### Step 2.1: Import FABRIC API and other libraries

In [ ]:
import json
import traceback

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();


### Step 2.2: Configure the parameters and variables
Can be used to set the corresponding slice and other variables used for subsequent cells.

In [ ]:
slice_name = 'DYNAMOS_EnergyEfficiency'
# Nodes:
node1_name = 'Node1'
node2_name = 'Node2'
node3_name = 'Node3'
# Network:
network_name = 'NET1'

## Step 3: Configure Network

### Step 3.1: Configure FABRIC Network for Nodes
This step configures the network by adding IP addresses to the nodes in the network so that they can communicate with each other over IPv4.

In [ ]:
try:
    # ========== Step 1: Get Slice and Network ==========
    # Get the slice
    # Get slice by name: https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.get_slice
    slice = fablib.get_slice(name=slice_name)
    # Get the network
    network = slice.get_network(name=network_name)
    # Get available ips
    network_available_ips = network.get_available_ips()
    # Print network
    print(f"{network}")

    # ========== Step 2: Configure Nodes and node interfaces ==========
    for node in slice.get_nodes():
        print(f"Node: {node.get_name()}")
        node_iface = node.get_interface(network_name=network_name)
        # Set IP address for this node (IPv4) to the first available ip address
        node_address = network_available_ips.pop(0)
        node_iface.ip_addr_add(addr=node_address, subnet=network.get_subnet())
        #node.ip_route_add(subnet=network.get_subnet(), gateway=network.get_gateway())
        
        # Print the IP to be used with Kubespray inventory
        print(f"IP for Kubespray inventory {node.get_name()}: {node_address}")

        # # Debug: Interface IP info and routing table
        # stdout, stderr = node.execute(f'ip addr show {node_iface.get_os_interface()}')
        # print(stdout)
        # stdout, stderr = node.execute(f'ip route list')
        # print(stdout)
    
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

### Step 3.2: Get SSH Commands
This step prints the SSH commands that can be used with the guide below to SSH into the VMs.

In [ ]:
# Print the necessary information
try:
    # Get slice nodes
    for node in slice.get_nodes():
        print(f"Node: {node.get_name()}")
        # Get the original SSH command
        original_ssh_command = node.get_ssh_command()
        # Print SSH commands to get into the nodes
        print(f"  SSH Command from FABRIC: {original_ssh_command}")
        # Replace the file paths in the SSH command
        updated_ssh_command = original_ssh_command.replace(
            "/home/fabric/work/fabric_config/slice_key", "~/.ssh/slice_key"
        ).replace(
            "/home/fabric/work/fabric_config/ssh_config", "ssh_config"
        )
        # Print the updated SSH command
        print(f"  SSH Command locally (ensuring it is saved according to below steps): {updated_ssh_command}")
    
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

#### Run the SSH Commands
To run the SSH Commands follow these steps:
1. From the Jupyter Notebook Hub from FABRIC, download the /fabric_config/fabric_bastion_key, /fabric_config/slice_key and /fabric_config/ssh_config files
2. Add the ssh_config file to this project under /fabric/fabric_config, and change the /fabric_config/ssh_config "IdentityFile" entry to "~/.ssh/fabric_bastion_key", this is the new path to the bastion key of FABRIC from where you will be running the file.

3. Execute these steps to save the SSH files savely on your local machine and avoid problems
```sh
# Open a Linux terminal, such as WSL after opening a CMD in Windows:
wsl
# Navigate to the Downloads directory
cd Downloads
# Create a directory called ssh to store the files
mkdir -p ~/.ssh

# Copy the key files to the SSH directory
cp slice_key ~/.ssh/slice_key
cp fabric_bastion_key ~/.ssh/fabric_bastion_key
# Update permissions
chmod 600 ~/.ssh/slice_key
chmod 600 ~/.ssh/fabric_bastion_key
# Navigate to the SSH directory to verify the files
cd ~/.ssh
# List files including permissions (-l)
ls -l

# Navigate to the fabric_config folder of this project, such as:
cd /mnt/c/Users/cpoet/VSC_Projs/EnergyEfficiency_DYNAMOS/fabric/fabric_config
# Then run the command from the previous step, such as:
ssh -i ~/.ssh/slice_key -F ssh_config ubuntu@2001:610:2d0:fabc:f816:3eff:fe65:a464
# To exit SSH access, type "exit" and press Enter
```
4. Now you can SSH into the nodes using the printed commands.

## Step 4: Configure Kubernetes Cluster with Kubespray

This step configures the Kubernetes cluster.

Go to the README.md in this folder of the GitHub project and follow the steps for "Configure Kubernetes cluster/environment". After these steps, you can do the next steps.

### Step 4.1: Post-install checks and configurations
After doing the above step, you can check the configuration by executing these steps. Note: do not forget to also upload the script file in Jupyter Hub for this part.

After the script execution: make sure to run "source ~/.bashrc" in any SSH session you have open to reload the PATH variables to be able to use the installations.


In [ ]:
try:
    # Get the control plane node (node1)
    node1 = slice.get_node(name=node1_name)
    # Upload script file to the node
    file_attributes = node1.upload_file(local_file_path="config_control_plane.sh", remote_file_path="config_control_plane.sh")
    # Add necessary permissions and execute the script
    stdout, stderr = node1.execute(f"chmod +x config_control_plane.sh && ./config_control_plane.sh")

except Exception as e:
    print(f"Exception: {e}")

### Step 4.2: Verify installations

Now you can move on to the next step to work with DYNAMOS in Kubernetes. Also, you can use the different Kubernetes tools similar to how you used to do locally now, such as k9s and etcd, etc., but now from the control plane node (in this case node1) by using SSH to log into the VM.

For example, test k9s by running "k9s" in the SSH connection to node1 and press "0" to see all namespaces to open k9s and see the different pods in all namespaces.